In [1]:
##
import os
import sys
import warnings
import enum
from enum import Enum
import matplotlib
from pathlib import Path
from typing import Callable

##
sys.path.append("./src/")
sys.path.append("./")
sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../../")

##
from finrl import config as finrl_config
from finrl.main import check_and_make_directories
from finrl.config_tickers import DOW_30_TICKER
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import (
    TEST_END_DATE,
    TRAINED_MODEL_DIR,
)
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.plot import backtest_stats, convert_daily_return_to_pyfolio_ts, get_baseline

##
from meta.data_processors.yahoofinance import Yahoofinance

/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/tensorboard/compat/proto/tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
/Users/zlapik/my-drive-zlapik/0-todo/ai-investing/venv3.10/lib/python3.10/site-packages/tensorboard/compat/proto/tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors 

In [2]:
def config():
    # #
    warnings.filterwarnings("ignore", category=UserWarning)  # TODO: zipline problem
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=RuntimeWarning)
    # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # INFO, WARN are ignored and only ERROR messages will be printed

    # #
    matplotlib.use("Agg")

    # #
    check_and_make_directories(
        [
            finrl_config.DATA_SAVE_DIR,
            finrl_config.TRAINED_MODEL_DIR,
            finrl_config.TENSORBOARD_LOG_DIR,
            finrl_config.RESULTS_DIR,
        ]
    )


config()

In [3]:
##
import numpy as np
import pandas as pd
from pyfolio import timeseries
import tqdm

##
from rl.data import DataTechnicalAnalysis, DataFundamentalAnalysis, DataBase
from common.utils import now_time
from configuration.settings import ProjectDir

In [4]:
prj_dir = ProjectDir(root=Path("/Users/zlapik/my-drive-zlapik/0-todo/ai-investing"))

In [5]:
_TRAIN_DATA_START = "2010-01-01"
_TRAIN_DATA_END = "2021-12-31"
_TEST_DATA_START = "2021-01-01"
_TEST_DATA_END = "2021-12-31"

In [6]:
# yahoo_data_api = Yahoofinance()
data_processor = DataBase(start_date=_TRAIN_DATA_START, end_date=_TEST_DATA_END, time_interval="1d")

In [7]:
data_processor.download_data(ticker_list=DOW_30_TICKER)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
print(data_processor.dataframe.shape)
data_processor.dataframe.head()

(88283, 9)


,date,open,high,low,close,adjusted_close,volume,tic,day
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.515213,493729600,AAPL,0
1,2010-01-04,56.630001,57.869999,56.560001,57.720001,43.267181,5277400,AMGN,0
2,2010-01-04,40.810001,41.099998,40.389999,40.919998,33.792702,6894300,AXP,0
3,2010-01-04,55.720001,56.389999,54.799999,56.180000,43.777554,6186700,BA,0
4,2010-01-04,57.650002,59.189999,57.509998,58.549999,41.353191,7325600,CAT,0


In [9]:
def check_dir(filename: Path):
    dir = filename.parent
    if os.path.isdir(dir):
        print(f"OK: Dir exist for: {filename}")
    else:
        raise NotADirectoryError(f"ERROR: Dir does not exist {dir}")


class FileType(Enum):
    JSON = enum.auto
    CSV = enum.auto


def save_dataset(dataframe: pd.DataFrame, type: FileType):
    print(f"Saving data into: {filename}")
    if type == FileType.JSON:
        dataframe.to_json(filename)
    elif type == FileType.CSV:
        dataframe.to_csv(filename)
    else:
        raise ValueError(f"type: FileType must be CSV or JSON.")
    print(f"Data saved to json: {filename}")


filename = data_processor.get_filename(prj_dir=prj_dir, name="dji30_ta")
check_dir(filename)

# %load_ext line_profiler
# %lprun -f save_dataset save_dataset()

OK: Dir exist for: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json


In [10]:
save_dataset(dataframe=data_processor.dataframe, type=FileType.JSON)

Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json
Data saved to json: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json


In [11]:
data_processor.dataframe.shape

(88283, 9)

In [12]:
fe = FeatureEngineer(use_technical_indicator=True, use_turbulence=False, user_defined_feature=False)
data_features: pd.DataFrame = fe.preprocess_data(data_processor.dataframe.copy())

Successfully added technical indicators


In [13]:
data_features.shape

(87580, 17)

In [14]:
save_dataset(dataframe=data_processor.dataframe, type=FileType.JSON)

Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json
Data saved to json: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json


In [15]:
def add_cov(df: pd.DataFrame) -> pd.DataFrame:
    ##
    # add covariance matrix as states
    df = df.sort_values(["date", "tic"], ignore_index=True)
    df.index = df.date.factorize()[0]

    cov_list = []
    return_list = []

    # look back is one year
    lookback = 252
    for i in tqdm.tqdm(range(lookback, len(df.index.unique()))):
        data_lookback = df.loc[i - lookback : i, :]
        price_lookback = data_lookback.pivot_table(index="date", columns="tic", values="close")
        return_lookback = price_lookback.pct_change().dropna()
        covs = return_lookback.cov().values

        #
        return_list.append(return_lookback)
        cov_list.append(covs)

    df_cov = pd.DataFrame({"date": df.date.unique()[lookback:], "cov_list": cov_list, "return_list": return_list})
    df = df.merge(df_cov, on="date")
    df = df.sort_values(["date", "tic"]).reset_index(drop=True)
    return df


data_features_cov1 = add_cov(data_features.copy())

100%|██████████| 2768/2768 [00:22<00:00, 123.76it/s]


In [16]:
print(data_features_cov1.shape)

(80272, 19)


In [17]:
# _df = data_features.copy()

# rolling_window =
# cov_matrix = _df.rolling(window=252).cov()

# print(rolling_window)
# print(_df.shape)

# data_features_cov2 = cov2(data_features.copy())

In [18]:
# print(data_features_cov2.shape)

In [19]:
def print_info(df: pd.DataFrame):
    ##
    print(f"{df.shape=}")
    print(f"Start date: {df['date'].iloc[0]}")
    print(f"End date: {df['date'].iloc[-1]}")
    print(f"Stocks: {df['tic'].unique().size}")

    ##
    data_grouped_by_date = df.groupby(by=["date"])
    grouped_data_size = data_grouped_by_date.size()
    if grouped_data_size.nunique() == 1:
        print(f"OK: All dates have same row count: {grouped_data_size.unique()}")
    else:
        raise ValueError(f"Different size for each date: {grouped_data_size.unique()}")


print_info(data_processor.dataframe)

df.shape=(88283, 9)
Start date: 2010-01-04
End date: 2021-12-30
Stocks: 30


ValueError: Different size for each date: [29 30]

In [25]:
data_features_cov1.head()

,date,open,high,low,close,adjusted_close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,cov_list,return_list
0,2011-01-03,11.630000,11.795000,11.601429,11.770357,10.033262,445138400,AAPL,0,0.118248,11.709899,11.326780,62.862180,144.286559,25.488753,11.415357,11.212333,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
1,2011-01-03,55.200001,56.279999,55.180000,55.549999,41.640541,5453300,AMGN,0,0.261720,58.568809,52.495191,50.863117,34.401812,4.927279,54.949000,55.565166,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
2,2011-01-03,43.299999,43.619999,43.110001,43.400002,36.488426,7633300,AXP,0,-0.178628,47.098241,41.260759,51.443736,-26.423899,13.049417,43.883000,42.473000,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
3,2011-01-03,66.150002,66.680000,66.000000,66.400002,53.034523,8072900,BA,0,-0.081580,66.520526,63.197475,51.659856,149.651676,11.924376,64.853001,66.948501,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...
4,2011-01-03,94.379997,94.809998,94.110001,94.150002,68.190575,5231500,CAT,0,2.266966,96.271975,88.886025,68.687627,82.330220,50.929600,90.157333,85.266833,"[[0.0002849618759954093, 9.196035527740299e-05...",tic AAPL AMGN AXP ...


In [27]:
_d = data_features_cov1["cov_list"].values
_d

array([array([[2.84961876e-04, 9.19603553e-05, 1.93065964e-04, 1.80283971e-04,
               2.07064983e-04, 2.11837588e-04, 1.68524329e-04, 1.25599231e-04,
               1.32380923e-04, 1.18833191e-04, 1.18015978e-04, 1.71944651e-04,
               1.15475971e-04, 1.55655983e-04, 6.45390087e-05, 1.50512933e-04,
               6.80812888e-05, 7.53141334e-05, 1.30623821e-04, 9.20004302e-05,
               1.32620322e-04, 1.35186921e-04, 6.39403639e-05, 9.69228066e-05,
               1.00050190e-04, 1.20487128e-04, 6.77860315e-05, 8.98269282e-05,
               5.08180241e-05],
              [9.19603553e-05, 2.01468864e-04, 1.21861520e-04, 1.09728155e-04,
               1.10547180e-04, 1.03487580e-04, 7.60569648e-05, 9.41976701e-05,
               9.72158101e-05, 8.51807635e-05, 8.12657592e-05, 1.08149076e-04,
               7.97360761e-05, 9.59204252e-05, 5.87404093e-05, 1.02985209e-04,
               5.83050390e-05, 6.30826143e-05, 7.52556248e-05, 8.59864504e-05,
               8.421

In [20]:
# check_dir()

save_dataset(dataframe=data_features_cov1, type=FileType.JSON)

# df_json = pd.read_json(filename, orient="split")
# df_json.head()

Saving data into: /Users/zlapik/my-drive-zlapik/0-todo/ai-investing/dataset/stock/ai4finance/dji30_ta_2022-12-14T20-13-55.json


KeyboardInterrupt: 